# OVC SERV _ MOTHER COMPONENT

#### The following dates are used to specify the periods for which we want to extract services data from HIV Haiti for PMTCT women. Feel free to modify them for the desired periods.

In [51]:
#Dates for the periods

#Dates for the most recent period
start_date = '2022-07-01'
end_date = '2022-09-30'
#Dates for the period before the most recent period
start_date_last = '2022-04-01'
end_date_last = '2022-06-30'

### Importing packages

In [62]:
#Importing packages

import pandas as pd
import numpy as np
from datetime import datetime
import pymysql
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
from dotenv import load_dotenv
import openpyxl

### Downloading Household mother file from Commcare

In [3]:
#Connecting to Commcare
load_dotenv('id_cc.env')
email = os.getenv('EMAIL')
password_cc = os.getenv('PASSWORD')

#Defining the driver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(1000)

#Creating login function
def commcare_login():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/form/download/269567f0b84da5a1767712e519ced62e/'
    )
    driver.find_element_by_xpath('//*[@id="id_auth-username"]').send_keys(email)
    driver.find_element_by_xpath('//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element_by_css_selector('button[type=submit]').click()
       
commcare_login()

#Download the database "Household mother"
driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

#-------------------------------------------------------------------------------------------------------------------------


True



====== WebDriver manager ======
Current google-chrome version is 106.0.5249
Get LATEST chromedriver version for 106.0.5249 google-chrome
Driver [C:\Users\Davidson Adrien\.wdm\drivers\chromedriver\win32\106.0.5249.61\chromedriver.exe] found in cache


In [67]:
#connecting to HIV Haiti
load_dotenv('dot.env')
user = os.getenv('MYSQL_USER')
password_hiv = f"{os.getenv('MYSQL_PASSWORD')}!"
host = os.getenv('MYSQL_HOST')
db = os.getenv('MYSQL_DB')

conn_text = f'mysql+pymysql://{user}:{password_hiv}@{host}/{db}'
engine = create_engine(conn_text)

True

In [68]:
password_hiv

'longlivecaris!'

### Connecting to HIV Haiti and downloading Mothers services data

In [4]:

# sql = f'''

# SELECT
#     bot.patient_code,
#     IF(p.id is null,"no","yes") as is_patient_on_hivhaiti,
#     IF(club_q1 IS NOT NULL,
#         'yes',
#         IF(quest_q1 IS NOT NULL,
#             'yes',
#             IF(arv_1 IS NOT NULL,
#                 'yes',
#                 IF(odk_1 IS NOT NULL,
#                     'yes',
#                     IF(ptme_1 IS NOT NULL,
#                         'yes',
#                         IF(ptme1_1 IS NOT NULL,
#                             'yes',
#                             IF(mereenfant_1 IS NOT NULL,
#                                 'yes',
#                                 IF(opf_q1.case_id IS NOT NULL ,'yes',
#                                 IF(tmf_q1.patient_code IS NOT NULL,'yes',
#                                 IF(tpv_q1.patient_code IS NOT NULL,"yes",
#                                 IF(oto1.patient_code IS NOT NULL,"yes",
#                                 'no'))))))))))) AS q1,
#     IF(club_q2 IS NOT NULL,
#         'yes',
#         IF(quest_q2 IS NOT NULL,
#             'yes',
#             IF(arv_2 IS NOT NULL,
#                 'yes',
#                 IF(odk_2 IS NOT NULL,
#                     'yes',
#                     IF(ptme_2 IS NOT NULL,
#                         'yes',
#                         IF(ptme1_2 IS NOT NULL,
#                             'yes',
#                             IF(mereenfant_2 IS NOT NULL,
#                                 'yes',
#                                 IF(opf_q2.case_id IS NOT NULL ,
#                                 'yes',
#                                 IF(tmf_q2.patient_code IS NOT NULL, 'yes',
#                                 IF(oto2.patient_code IS NOT NULL, 'yes',
#                                 IF(tpv_q2.patient_code IS NOT NULL,'yes','no'))))))))))) AS q2,
#     IF(club_q1 IS NOT NULL, 'yes', 'no') AS club_q1,
#     IF(club_q2 IS NOT NULL, 'yes', 'no') AS club_q2,
#     IF(quest_q1 IS NOT NULL, 'yes', 'no') AS quest_q1,
#     IF(quest_q2 IS NOT NULL, 'yes', 'no') AS quest_q2,
#     IF(arv_1 IS NOT NULL, 'yes', 'no') AS on_arv_q1,
#     IF(arv_2 IS NOT NULL, 'yes', 'no') AS on_arv_q2,
#     IF(odk_1 IS NOT NULL, 'yes', 'no') AS odk_q1,
#     IF(odk_2 IS NOT NULL, 'yes', 'no') AS odk_q2,
#     IF(tmf_q1.patient_code IS NOT NULL, 'yes',"no")  as followup_q1,
#     IF(tmf_q2.patient_code IS NOT NULL, 'yes',"no")  as followup_q2,
#     IF(tpv_q1.patient_code IS NOT NULL,'yes','no') as visit_ptme_q1,
#     IF(tpv_q2.patient_code IS NOT NULL,'yes','no') as visit_ptme_q2,
#     IF(oto1.patient_code IS NOT NULL,'yes','no') as visit_ptme_ratio_q1,
#     IF(oto2.patient_code IS NOT NULL,'yes','no') as visit_ptme_ratio_q2,

#     IF(opf_q1.case_id IS NOT NULL ,'yes','no') AS odk_phone_followup_q1,
#     IF(opf_q2.case_id IS NOT NULL ,'yes','no') AS odk_phone_followup_q2,
#     if(tmf_q1.patient_code IS NOT NULL,'yes','no') as mother_followup_q1,
#     if(tmf_q2.patient_code IS NOT NULL,'yes','no') as mother_followup_q2,
#     IF(COALESCE(ptme_1, ptme1_1) IS NOT NULL,
#         'yes',
#         'no') AS ptme_q1,
#     IF(COALESCE(ptme_2, ptme1_2) IS NOT NULL,
#         'yes',
#         'no') AS ptme_q2,
#     IF(mereenfant_1 IS NOT NULL,
#         'yes',
#         'no') AS mereenfant_q1,
#     IF(mereenfant_2 IS NOT NULL,
#         'yes',
#         'no') AS mereenfant_2,
#     office,
#     p.created_at as patient_created_date,
#     IF(p.created_at BETWEEN '{start_date}' AND '{end_date}' , "yes","no") as is_patient_created_on_q2,
#     IF(tr.id_patient is not null , "yes","no") as on_arv,
#     tmi.dob,
#     tmi.is_dead
# FROM
#     (SELECT
#         *
#     FROM
#         (SELECT
#         patient_code
#     FROM
#         tracking_motherbasicinfo
#     LEFT JOIN patient ON patient.id = tracking_motherbasicinfo.id_patient) o UNION (SELECT
#         health_id AS patient_code
#     FROM
#         openfn.odk_pregnancy_visit
#     WHERE
#         date_of_visit BETWEEN '{start_date_last}' AND '{end_date}') 
#         ## ODK PHONE FOLLOWUP START
#         UNION (
#         select topf.patient_code as patient_code from tracking_odk_phone_followup topf 
# 			where topf.eccm_joignable_par_tel!=0 and (topf.eccm_date between '{start_date_last}' AND '{end_date}') and topf.name='Enquette Corona club meres'
#         )
#         ## ODK PHONE FOLLOWUP END

#         UNION (

#           select tpv.patient_code from tracking_ptme_visit tpv where tpv.date_of_visit between '{start_date_last}' AND '{end_date}'
        
#         )
#         ## ODK Ratio Visits
#         UNION (
#            select opt.patient_code from odk_tracking_other_visit_ptme opt where opt.date_of_visit between '{start_date_last}' AND '{end_date}' 
#         )
#         ) bot
#         LEFT JOIN
#     (SELECT DISTINCT
#         patient_code AS club_q2
#     FROM
#         session
#     LEFT JOIN club_session ON club_session.id = session.id_club_session
#     LEFT JOIN patient ON patient.id = session.id_patient
#     WHERE
#         is_present = 1
#             AND club_session.date BETWEEN '{start_date}' AND '{end_date}') c2 ON club_q2 = bot.patient_code
#         LEFT JOIN
#     patient ON patient.patient_code = bot.patient_code
#         LEFT JOIN
#     (SELECT DISTINCT
#         patient_code AS club_q1
#     FROM
#         session
#     LEFT JOIN club_session ON club_session.id = session.id_club_session
#     LEFT JOIN patient ON patient.id = session.id_patient
#     WHERE
#         is_present = 1
#             AND club_session.date BETWEEN '{start_date_last}' AND '{end_date_last}') c1 ON club_q1 = bot.patient_code
#         LEFT JOIN
#     (SELECT
#         patient_code AS quest_q1, date
#     FROM
#         (SELECT
#         id_patient, date
#     FROM
#         questionnaire_motherhivknowledge UNION SELECT
#         id_patient, date
#     FROM
#         questionnaire_mothersurvey UNION SELECT
#         id_patient, date
#     FROM
#         questionnaire_newmotherhivknowledge) x
#     LEFT JOIN patient ON patient.id = x.id_patient
#     WHERE
#         date BETWEEN '{start_date_last}' AND '{end_date_last}') quest ON quest_q1 = bot.patient_code
#         LEFT JOIN
#     (SELECT
#         patient_code AS quest_q2
#     FROM
#         (SELECT
#         id_patient
#     FROM
#         questionnaire_motherhivknowledge
#     WHERE
#         date BETWEEN '{start_date}' AND '{end_date}' UNION SELECT
#         id_patient
#     FROM
#         questionnaire_mothersurvey
#     WHERE
#         date BETWEEN '{start_date}' AND '{end_date}' UNION SELECT
#         id_patient
#     FROM
#         questionnaire_newmotherhivknowledge
#     WHERE
#         date BETWEEN '{start_date}' AND '{end_date}') y
#     LEFT JOIN patient ON patient.id = y.id_patient) ques ON quest_q2 = bot.patient_code
#         LEFT JOIN
#     lookup_hospital ON CONCAT(lookup_hospital.city_code,
#             '/',
#             lookup_hospital.hospital_code) = LEFT(patient.patient_code, 8)
#         LEFT JOIN
#     (SELECT DISTINCT
#         patient_code AS arv_1
#     FROM
#         tracking_regime
#     LEFT JOIN patient ON patient.id = id_patient
#     WHERE
#         (start_date BETWEEN '{start_date_last}' AND '{end_date_last}'
#             OR end_date BETWEEN '{start_date_last}' AND '{end_date_last}')
#             AND category = 'regime_mother_treatment') ar ON arv_1 = bot.patient_code
#         LEFT JOIN
#     (SELECT DISTINCT
#         patient_code AS arv_2
#     FROM
#         tracking_regime
#     LEFT JOIN patient ON patient.id = id_patient
#     WHERE
#         (start_date BETWEEN '{start_date}' AND '{end_date}'
#             OR end_date BETWEEN '{start_date}' AND '{end_date}')
#             AND category = 'regime_mother_treatment') arv ON arv_2 = bot.patient_code
#         LEFT JOIN
#     (SELECT DISTINCT
#         health_id AS odk_1
#     FROM
#         openfn.odk_pregnancy_visit
#     WHERE
#         date_of_visit BETWEEN '{start_date_last}' AND '{end_date_last}') odk1 ON odk_1 = bot.patient_code
#         LEFT JOIN
#     (SELECT DISTINCT
#         health_id AS odk_2
#     FROM
#         openfn.odk_pregnancy_visit
#     WHERE
#         date_of_visit BETWEEN '{start_date}' AND '{end_date}') odk2 ON odk_2 = bot.patient_code
#         LEFT JOIN
#     (SELECT DISTINCT
#         patient_code AS ptme_1
#     FROM
#         tracking_pregnancy
#     LEFT JOIN patient ON patient.id = id_patient_mother
#     WHERE
#         (ptme_enrollment_date BETWEEN '{start_date_last}' AND '{end_date_last}')
#             OR (actual_delivery_date BETWEEN '{start_date_last}' AND '{end_date_last}')) ptme1 ON ptme_1 = bot.patient_code
#         LEFT JOIN
#     (SELECT DISTINCT
#         patient_code AS ptme_2
#     FROM
#         tracking_pregnancy
#     LEFT JOIN patient ON patient.id = id_patient_mother
#     WHERE
#         (ptme_enrollment_date BETWEEN '{start_date}' AND '{end_date}')
#             OR (actual_delivery_date BETWEEN '{start_date}' AND '{end_date}')) ptme2 ON ptme_2 = bot.patient_code
#         LEFT JOIN
#     (SELECT DISTINCT
#         patient_code AS ptme1_1
#     FROM
#         tracking_motherbasicinfo
#     LEFT JOIN patient ON patient.id = id_patient
#     WHERE
#         PTME_date BETWEEN '{start_date_last}' AND '{end_date_last}') pt1 ON ptme1_1 = bot.patient_code
#         LEFT JOIN
#     (SELECT DISTINCT
#         patient_code AS ptme1_2
#     FROM
#         tracking_motherbasicinfo
#     LEFT JOIN patient ON patient.id = id_patient
#     WHERE
#         PTME_date BETWEEN '{start_date}' AND '{end_date}') pt2 ON ptme1_2 = bot.patient_code
#         LEFT JOIN
#     (SELECT
#         patient.patient_code AS mereenfant_1
#     FROM
#         tracking_motherbasicinfo
#     LEFT JOIN patient ON patient.id = tracking_motherbasicinfo.id_patient
#     LEFT JOIN testing_mereenfant ON CONCAT(testing_mereenfant.mother_city_code, '/', testing_mereenfant.mother_hospital_code, '/', testing_mereenfant.mother_code) = patient_code
#     WHERE
#         date BETWEEN '{start_date_last}' AND '{end_date_last}'
#             AND patient_code IS NOT NULL
#     GROUP BY patient_code) me1 ON mereenfant_1 = bot.patient_code
#         LEFT JOIN
#     (SELECT
#         patient.patient_code AS mereenfant_2
#     FROM
#         tracking_motherbasicinfo
#     LEFT JOIN patient ON patient.id = tracking_motherbasicinfo.id_patient
#     LEFT JOIN testing_mereenfant ON CONCAT(testing_mereenfant.mother_city_code, '/', testing_mereenfant.mother_hospital_code, '/', testing_mereenfant.mother_code) = patient_code
#     WHERE
#         date BETWEEN '{start_date}' AND '{end_date}'
#             AND patient_code IS NOT NULL
#     GROUP BY patient_code) me2 ON mereenfant_2 = bot.patient_code
#     left join patient p on p.patient_code=bot.patient_code
#     ## ODK FOLLOWUP PHONE END
# 	LEFT JOIN (
#     select case_id,eccm_date, topf.patient_code from tracking_odk_phone_followup topf
#     where topf.eccm_joignable_par_tel!=0 and topf.eccm_date BETWEEN '{start_date_last}' AND '{end_date_last}' group by topf.case_id
#     ) opf_q1 on opf_q1.patient_code= bot.patient_code
# 	LEFT JOIN (
#     select case_id,eccm_date, topf.patient_code from tracking_odk_phone_followup topf
#     where topf.eccm_joignable_par_tel!=0 and topf.eccm_date BETWEEN '{start_date}' AND '{end_date}' group by topf.case_id
#     ) opf_q2 on opf_q2.patient_code= bot.patient_code
    
# 	## MOTHER FOLLOWUP  START
# 	LEFT JOIN (
#     select tmf.date, p.patient_code from tracking_motherfollowup tmf
#     left join patient p on p.id=tmf.id_patient
#     where  tmf.date BETWEEN '{start_date_last}' AND '{end_date_last}' group by tmf.id_patient
#     ) tmf_q1 on tmf_q1.patient_code= bot.patient_code
# 	LEFT JOIN (
#     select tmf.date, p.patient_code from tracking_motherfollowup tmf
#     left join patient p on p.id=tmf.id_patient
#     where  tmf.date BETWEEN '{start_date}' AND '{end_date}' group by tmf.id_patient
#     ) tmf_q2 on tmf_q2.patient_code= bot.patient_code
    
#     ##  MOTHER FOLLOWUP END

# 	## MOTHER Agent visit ptme  START
# 	LEFT JOIN (
#     select tpv.date_of_visit, tpv.patient_code from tracking_ptme_visit tpv
#     where  tpv.date_of_visit BETWEEN '{start_date_last}' AND '{end_date_last}' group by tpv.patient_code
#     ) tpv_q1 on tpv_q1.patient_code= bot.patient_code
# 	LEFT JOIN (
#     select tpv.date_of_visit, tpv.patient_code from tracking_ptme_visit tpv
#     where  tpv.date_of_visit BETWEEN '{start_date}' AND '{end_date}' group by tpv.patient_code
#     ) tpv_q2 on tpv_q2.patient_code= bot.patient_code
    
#     ##  MOTHER Agent visit ptme END
    
#     ## MOTHER ratio start
    
#     LEFT JOIN (
#     select oto11.date_of_visit, oto11.patient_code from odk_tracking_other_visit_ptme oto11 
#     where oto11.date_of_visit BETWEEN '{start_date_last}' AND '{end_date_last}' group by oto11.patient_code
#     ) oto1 on oto1.patient_code= bot.patient_code
# 	LEFT JOIN (
#     select oto21.date_of_visit, oto21.patient_code from odk_tracking_other_visit_ptme oto21 
#     where oto21.date_of_visit BETWEEN '{start_date}' AND '{end_date}' group by oto21.patient_code
#     ) oto2 on oto2.patient_code= bot.patient_code
    
#     ## MOTHER ratio end
#     left join tracking_motherbasicinfo tmi on tmi.id_patient=p.id
#     left join (select distinct id_patient from tracking_regime where category='regime_mother_treatment' and (id_arv is not null) and id_arv!=0  ) tr  on tr.id_patient=p.id
# WHERE
#     (p.linked_to_id_patient = 0 or p.linked_to_id_patient is null)
#         AND (club_q1 IS NOT NULL
#         OR club_q2 IS NOT NULL
#         OR quest_q1 IS NOT NULL
#         OR quest_q2 IS NOT NULL
#         OR arv_1 IS NOT NULL
#         OR arv_2 IS NOT NULL
#         OR odk_1 IS NOT NULL
#         OR odk_2 IS NOT NULL
#         OR ptme_1 IS NOT NULL
#         OR ptme_2 IS NOT NULL
#         OR ptme1_1 IS NOT NULL
#         OR ptme1_2 IS NOT NULL
#         OR mereenfant_1 IS NOT NULL
#         OR mereenfant_2 IS NOT NULL
#         OR opf_q1.case_id IS NOT NULL
#         OR opf_q2.case_id IS NOT NUlL
#         OR tmf_q1.patient_code IS NOT NULL
#         OR tmf_q2.patient_code IS NOT NULL
#         OR tpv_q1.patient_code IS NOT NULL 
#         OR tpv_q2.patient_code IS NOT NULL
#         OR oto1.patient_code IS NOT NULL
#         OR oto2.patient_code IS NOT NULL
        
#         )
# GROUP BY bot.patient_code

# '''

# #Reading the OVC Helper Mother file
# data = pd.read_sql_query(sql, engine, parse_dates = True)
# data.shape


True

(5109, 32)

In [8]:
# data.to_excel('PTME_served_Q4FY22.xlsx', index = False)

## Importing services database

In [45]:
data = pd.read_excel("C:/Users/Davidson Adrien/Documents/Monitoring & Evaluation/Senior M&E files/DATIM_FY22/Q4FY22/PTME/ovc_helper_mother_2022-10-12.xlsx", parse_dates = True)

In [41]:
data.shape

(4931, 24)

### Preparing OVC data

In [46]:

#Importing dataset with hospital name, department, commune, networks name
infos = pd.read_excel('Caris Sites_modified.xlsx', usecols = ['CODE LABO', 'HOPITAL', 'RESEAU','DEPT', 'COMMUNE'],sheet_name = 'Caris Sites')
infos.rename(columns = {'CODE LABO': 'site',
                       'HOPITAL': 'hospital',
                       'DEPT': 'Department',
                       'COMMUNE': 'Commune',
                       'RESEAU': 'network'}, inplace = True)

#Creating the sites column to merge with the other datasets
data['site'] = data['patient_code'].apply(lambda x: x[0:8])
data['site'].replace(['PAP/CSPP', 'PAP/SADA', 'PAP/OMPF'], ['STM/CSPP', 'ARC/SADA', 'PAP/OMFP'], inplace = True)

#converting dob column into datetime type
data['dob'] = pd.to_datetime(data['dob'], errors = 'coerce')

#Adding some required columns
data_merge = data.merge(infos, on = 'site', how = 'left')

#filtering for patient on HIV Haiti and not belonging to UGP
data_no_UGP = data_merge[data_merge['network'] != 'UGP']
data_on_hivhaiti = data_no_UGP[data_no_UGP['is_patient_on_hivhaiti'] != 'no']

#Replacing some irrelevant dates of birth
data_on_hivhaiti['dob'].fillna('1901-01-01', inplace = True)
data_on_hivhaiti.loc[(data_on_hivhaiti.dob == '1775-01-01'), 'dob'] = '1901-01-01'
# data_on_hivhaiti.loc[(data_on_hivhaiti.dob == '1079-06-15'), 'dob'] = '1901-01-01'
data_on_hivhaiti.loc[(data_on_hivhaiti.dob == '1297-07-01'), 'dob'] = '1901-01-01'
data_on_hivhaiti.loc[(data_on_hivhaiti.dob == '1086-06-18'), 'dob'] = '1901-01-01'
# data_on_hivhaiti.loc[(data_on_hivhaiti.dob == '1898-03-22'), 'dob'] = '1901-01-01'
data_on_hivhaiti.loc[(data_on_hivhaiti.dob == '1089-10-05'), 'dob'] = '1901-01-01'
data_on_hivhaiti.loc[(data_on_hivhaiti.dob == '1089-02-14'), 'dob'] = '1901-01-01'


#Transforming dob into datetime type
data_on_hivhaiti['dob'] = pd.to_datetime(data_on_hivhaiti['dob'], errors = 'coerce')

#Creating the age column
def get_age(date):
        return relativedelta(parse(end_date), date).years

data_on_hivhaiti['age'] = data_on_hivhaiti['dob'].map(get_age)

ovc_data = data_on_hivhaiti

ovc_data.head(2)

,patient_code,on_arv,is_dead,is_abandoned,office,dob,q1,club_q1,quest_q1,on_arv_q1,...,club_q2,odk_phone_followup_q2,graduation,is_patient_on_hivhaiti,site,hospital,network,Department,Commune,age
3,ARC/SADA/ST000357,no,1,0,PAP,1970-06-17,yes,no,no,no,...,no,no,---,yes,ARC/SADA,SADA Matheux,Santé,Ouest,Arcahaie,52
4,ARC/SADA/ST000575,yes,0,0,PAP,1991-05-12,no,no,no,yes,...,no,no,---,yes,ARC/SADA,SADA Matheux,Santé,Ouest,Arcahaie,31


In [29]:
# data_merge.to_excel('Test_data_10122022.xlsx', index = False)

In [47]:
#Served women for the 1st semester

#-------------------------------------------------------------------------------------------------------------------------

#served women in Q1 and Q2
q1q2 = ovc_data[(ovc_data.q1 == 'yes') & (ovc_data.q2 == 'yes')][['patient_code', 'club_q1', 'club_q2', 'on_arv', 'Commune', 'age']]

#served women created while Q2
q2 = ovc_data[(ovc_data.patient_created_date >= start_date) & (ovc_data.patient_created_date <= end_date)][['patient_code', 'club_q1', 'club_q2', 'on_arv', 'Commune', 'age']]

#Concatenation of the two previous datasets
OVC = pd.concat([q1q2, q2]).drop_duplicates('patient_code').reset_index(drop = True)
print(f"\nNumber of women (OVC-)served for the first semester (including mothers aged less than 18): {OVC.shape[0]}.")

#======================================================================================================================

OVC.loc[(OVC.patient_code == 'CAP/KEND/ST00119'), 'Commune'] = 'Cap-Haitien'
OVC.loc[(OVC.patient_code == 'FOS/CFLC/ST00037'), 'Commune'] = 'Port-au-Prince'
OVC.loc[(OVC.patient_code == 'FOS/CFLC/ST00215'), 'Commune'] = 'Port-au-Prince'
OVC.loc[(OVC.patient_code == 'FOS/CFLC/ST00562'), 'Commune'] = 'Port-au-Prince'
OVC.loc[(OVC.patient_code == 'FOS/CFLC/ST00573'), 'Commune'] = 'Port-au-Prince'
OVC.loc[(OVC.patient_code == 'PDP/HSBB/ST00696'), 'Commune'] = 'Port-de-Paix'
OVC.loc[(OVC.patient_code == 'CAP/FOLF/ST03033'), 'Commune'] = 'Cap-Haitien'
OVC.loc[(OVC.patient_code == 'CAP/HOLF/ST03237'), 'Commune'] = 'Cap-Haitien'

#=======================================================================================================================
OVC.head()


Number of women (OVC-)served for the first semester (including mothers aged less than 18): 3763.


,patient_code,club_q1,club_q2,on_arv,Commune,age
0,ARC/SADA/ST00091,no,yes,yes,Arcahaie,34
1,ARC/SADA/ST00121,yes,yes,yes,Arcahaie,34
2,ARC/SADA/ST00144,yes,yes,yes,Arcahaie,35
3,ARC/SADA/ST00243,no,no,yes,Arcahaie,47
4,ARC/SADA/ST00265,no,no,yes,Arcahaie,38


In [48]:
OVC.to_excel("OVC_mothers_FY22Q4_10122022.xlsx", index = False)

In [69]:
#Importing household mother file
hh_mother = pd.read_excel('C:/Users/Davidson Adrien/Downloads/Ajout de menages ptme [officiel] ' + str(datetime.today().strftime('%Y-%m-%d')) + '.xlsx', 
                    sheet_name = 'Repeat- household_mother'
                    , parse_dates = True)

#===================================================================================================================
#Changing some columns name
hh_mother.rename(columns = {'form.household_mother.Full_code_patient_menage': 'full_code_patient_menage',
                           'form.household_mother.health_id':'patient_code',
                           'form.household_mother.age_in_year': 'age_in_year',
                           'form.household_mother.gender':'gender',
                           'form.household_mother.ptme_relationship':'ptme_relationship',
                           'form.household_mother.hiv_test_result': 'hiv_test_result'}, inplace = True)

#====================================================================================================================

hh_mother.full_code_patient_menage = hh_mother.full_code_patient_menage.str.upper()

#Importing All patient table from HIV Haiti to know whether the children code is on HIV Haiti or not
sql2 = 'select * from patient'
all_patient = pd.read_sql_query(sql2, engine, parse_dates = True)

#Regime ARV for kids
sql3 = '''
SELECT 
        p.patient_code, id_patient,
            MIN(IF(start_date != '0000-00-00', start_date, NULL)) AS start_date,
            GROUP_CONCAT(la.acronym
                SEPARATOR '-') AS regime,
            GROUP_CONCAT(tr.start_date
                SEPARATOR ' | ') AS all_start_date,
            GROUP_CONCAT(tr.end_date
                SEPARATOR ' | ') AS all_end_date
    FROM
        tracking_regime tr
	LEFT JOIN patient p ON p.id = tr.id_patient
    LEFT JOIN lookup_arv la ON la.id = tr.id_arv
    WHERE
        category = 'regime_infant_treatment' and tr.end_date='0000-00-00'
    GROUP BY id_patient
'''

regime = pd.read_sql_query(sql3, engine, parse_dates = True)
regime.rename(columns = {'patient_code': 'full_code_patient_menage',
                        'start_date': 'arv_start_date'}, 
              inplace = True)

#=========================================================================================================================
#List of women in club for the semester
club_ptme = OVC[(OVC.club_q1 == 'yes') | (OVC.club_q2 == 'yes')][['patient_code']]
print(f"Number of women in club for the semester: {club_ptme.shape[0]}")

Number of women in club for the semester: 2568


In [70]:
club_ptme.to_excel('Femmes_servies_en_club_Q2FY22.xlsx', index = False)

In [71]:
#Function to get the age range
def age_range(age):
    if age < 1:
        return '< 1'
    elif age in range(1, 5):
        return '1 - 4'
    elif age in range(5, 10):
        return '5 - 9'
    elif age in range(10, 15):
        return '10 - 14'
    elif age in range(15, 18):
        return '15 - 17'
    else:
        return '18+'

#----------------------------------------------------------------------------------------------------------------------
#Function to know whether a patient code is on HIV Haiti or not
def on_hivhaiti(code):
    if code.upper() in all_patient.patient_code.tolist():
        return 'yes'
    else:
        return 'no'
    
#----------------------------------------------------------------------------------------------------------------------

#data for only mothers in club
hhm_club = hh_mother[hh_mother['patient_code'].str.lower().isin(club_ptme['patient_code'].str.lower())]
hhm_club['patient_code'] = hhm_club['patient_code'].str.upper()

#preparing age columns
hhm_club.loc[(hhm_club.age_in_year == '---'), 'age_in_year'] = ''
hhm_club['age_in_year'] = pd.to_numeric(hhm_club['age_in_year'])
hhm_club['age_range'] = hhm_club['age_in_year'].map(age_range)

#column specifying if the code is on HIV Haiti
hhm_club['on_HIVHAITI'] = hhm_club['full_code_patient_menage'].map(on_hivhaiti)

#Replacing code by true gender values
hhm_club.loc[(hhm_club.gender == str(1)), 'gender'] = 'Male'
hhm_club.loc[(hhm_club.gender == str(2)), 'gender'] = 'Female'

#Adding the communes column
hhm_club = hhm_club.merge(OVC[['patient_code', 'Commune']], on = 'patient_code', how = 'left')
hhm_club.head(2)

,number,number__0,number__1,form.household_mother.last_name,form.household_mother.first_name,form.household_mother.dob,gender,form.household_mother.is_caris_beneficiary,form.household_mother.cty,form.household_mother.hospital_code,...,age_in_year,form.household_mother.city_code,form.household_mother.hospital_cd,full_code_patient_menage,form.household_mother.st_cod,patient_code,form.household_mother.case.@case_id,age_range,on_HIVHAITI,Commune
0,7.0,7,0,charles,louise Marie,1993-01-09,Female,1,cap,Hscm,...,27,cap,Hscm,CAP/HSCM/ST03435,st03435,CAP/HSCM/ST03435,bf0f06ff-31ff-46cc-a89a-b998c42c1de8,18+,yes,Milot
1,7.1,7,1,Dorsainvil,jacson,2002-08-17,Male,2,---,---,...,18,NaN,NaN,//,NaN,CAP/HSCM/ST03435,8a893ad8-6495-4725-847a-5cf02a71ee73,18+,no,Milot


In [72]:
#creating the mothers kids dataset and the dataset for the other kids
hhm_ff = hhm_club[hhm_club.ptme_relationship == str(2)]
hhm_other = hhm_club[hhm_club.ptme_relationship != str(2)]

#Size
print(f"Number of direct kids: {hhm_ff.shape[0]}.")
print(f"Number of other kids: {hhm_other.shape[0]}.")

Number of direct kids: 6866.
Number of other kids: 4800.


In [73]:
#kids less than 18
hhm_ff_17 = hhm_ff[hhm_ff['age_range'] != '18+']
hhm_other_17 = hhm_other[hhm_other['age_range'] != '18+']

#Size
print(f"Number of direct kids less than 18: {hhm_ff_17.shape[0]}.")
print(f"Number of other kids less than 18: {hhm_other_17.shape[0]}.")

Number of direct kids less than 18: 6480.
Number of other kids less than 18: 1555.


In [74]:
hhm_ff.to_excel("direct_kids_fy22q2.xlsx", index = False)
hhm_other.to_excel("indirect_kids_fy22q2.xlsx", index = False)

# DATIM Direct kids

In [75]:
#Creating Datim Table for mothers' kids
index_ff = hhm_ff.pivot_table(index = 'Commune', columns = ['gender', 'age_range'], values = 'patient_code', 
                              aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')
columns = [('Female', '< 1'),
 ('Female', '1 - 4'),
 ('Female', '5 - 9'),
 ('Female', '10 - 14'),
 ('Female', '15 - 17'),
 ('Female', '18+'),
 ('Male', '< 1'),
 ('Male', '1 - 4'),
 ('Male', '5 - 9'),
 ('Male', '10 - 14'),
 ('Male', '15 - 17'),
 ('Male', '18+')]
index_ff = index_ff.reindex(columns = columns)
index_ff[('Female', '18+')] = 0
index_ff[('Male', '18+')] = 0
index_ff.reset_index(inplace = True)
index_ff

gender                 Commune Female                                 Male  \
age_range                         < 1 1 - 4 5 - 9 10 - 14 15 - 17 18+  < 1   
0              Anse D'Hainault      2     5     3       2       1   0    2   
1                   Anse-Rouge      0     1     5       0       1   0    1   
2                Anse-a-Galets      7    27    25      12      11   0    4   
3                  Anse-a-Veau      3     3     3       3       0   0    3   
4                        Aquin      4     5    10       7       3   0    6   
5                     Arcahaie     21    51    54      39      16   0   22   
6                  Bassin Bleu      2    13     7       5       1   0    9   
7                  Cap-Haitien     16    30    23      16       4   0   16   
8                      Caracol      4     4     8       4       2   0    1   
9                    Carrefour     12    16    20      26       6   0   15   
10                   Cavaillon      4    13     9       3       2   0    9   
11                       Cayes      9    15    18      11       5   0   14   
12          Croix-des-Bouquets      6    51    58      25       9   0    6   
13                      Delmas     38    75   128     154      44   0   28   
14                  Dessalines     20    39    37      36       9   0   31   
15                      Ennery     17    27     8       9       4   0   14   
16                    Gonaives     20    20    23      15       4   0   21   
17                  Gros-Morne      7    21    42      26       1   0   19   
18                      Hinche      0     4     2       1       0   0    1   
19                 Ile-a-Vache      0     5     2       0       0   0    3   
20                     Jeremie     19    17    21       9       3   0    7   
21                    Kenscoff      8    13    16      17       6   0   10   
22                     L'Asile      2     8     5       5       1   0    6   
23                    L'Estere      4     2     5       3       1   0    5   
24                   La Tortue      2     1     2       0       0   0    1   
25                     Leogane      6    13     6       8       0   0   10   
26                 Les Anglais      0     0     2       2       1   0    0   
27                       Limbe      1     8     9       7       2   0    2   
28                   Marmelade     11    16    11       6       0   0    5   
29                       Milot      7    14    11      25      10   0    5   
30                   Miragoane     11    16    18      12       5   0   16   
31               Mont-Organise      0     0     3       0       0   0    0   
32                       Moron      5     3     6       5       1   0    5   
33                 Ouanaminthe     11     6    11       8       1   0    6   
34                 Petit-Goave      8    17     7      10       4   0    9   
35                  Petit-Trou      0     4     4       1       0   0    5   
36              Petite-Riviere      9     1     0       0       0   0   14   
37                      Pignon     10    16    13      15       4   0    7   
38                      Pilate      6    13    19      17       6   0   10   
39              Plaine-du-Nord      7     8    10      11       2   0    8   
40                 Port-Margot      2     2     4       4       1   0    4   
41              Port-au-Prince     71   177   154     180      47   0   52   
42                Port-de-Paix     14    28    32      26      13   0   16   
43         Saint-Louis-du-Nord      5    10     8       4       2   0    9   
44                  Saint-Marc     21    15     8       9       3   0   15   
45               Saint-Raphael      2     5    10      10       2   0   10   
46                     Tabarre     13    16    27      35       7   0   10   
47               Terrier-Rouge      6    10    11       8       1   0    3   
48                Trou-du-Nord      9    11    12      13       1   0   21   
49                    

## OVC_HIVSTAT _ Mothers kids

In [76]:
#Negative children
hhm_negative_children = hhm_ff[(hhm_ff.hiv_test_result == 'negatif') & (hhm_ff.age_in_year <= 17)]
negative_children = hhm_negative_children.pivot_table(index = 'Commune', values = 'patient_code', 
                                                      columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len, fill_value = 0)
negative_children.rename(columns = {'negatif': 'negative'}, inplace = True)

columns = [('negative','Female', '< 1'),
 ('negative','Female', '1 - 4'),
 ('negative','Female', '5 - 9'),
 ('negative','Female', '10 - 14'),
 ('negative','Female', '15 - 17'),
 ('negative','Male', '< 1'),
 ('negative','Male', '1 - 4'),
 ('negative','Male', '5 - 9'),
 ('negative','Male', '10 - 14'),
 ('negative','Male', '15 - 17')]

negative_children = negative_children.reindex(columns = columns)
negative_children.columns = ['_'.join(col) for col in negative_children.columns.values]
negative_children.reset_index(inplace = True)

negative_children.head()

,Commune,negative_Female_< 1,negative_Female_1 - 4,negative_Female_5 - 9,negative_Female_10 - 14,negative_Female_15 - 17,negative_Male_< 1,negative_Male_1 - 4,negative_Male_5 - 9,negative_Male_10 - 14,negative_Male_15 - 17
0,Anse D'Hainault,0,2,0,1,0,1,2,1,0,0
1,Anse-Rouge,0,0,1,0,0,0,0,1,1,0
2,Anse-a-Galets,5,26,25,12,10,4,31,14,18,5
3,Anse-a-Veau,0,2,2,2,0,2,5,2,4,1
4,Aquin,3,3,9,5,2,5,8,10,3,4


In [77]:
#Positive children
def on_arv(date):
    if pd.notna(date):
        return 'yes'
    else:
        return 'no'


hhm_positive_children = hhm_ff[(hhm_ff.hiv_test_result == 'positif') & (hhm_ff.on_HIVHAITI == 'yes') & (hhm_ff.age_in_year <= 17)].drop_duplicates('full_code_patient_menage')
hhm_positive_children = hhm_positive_children.merge(regime[['full_code_patient_menage', 'arv_start_date']], 
                                                    on = 'full_code_patient_menage', how = 'left')
hhm_positive_children['on_arv'] = hhm_positive_children.arv_start_date.map(on_arv)
positive_children = hhm_positive_children.pivot_table(values = 'patient_code', index = 'Commune', 
                                                      columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len, fill_value = 0)
positive_children.rename(columns = {'positif': 'positive'}, inplace = True)

columns = [('positive','Female', '< 1'),
 ('positive','Female', '1 - 4'),
 ('positive','Female', '5 - 9'),
 ('positive','Female', '10 - 14'),
 ('positive','Female', '15 - 17'),
 ('positive','Male', '< 1'),
 ('positive','Male', '1 - 4'),
 ('positive','Male', '5 - 9'),
 ('positive','Male', '10 - 14'),
 ('positive','Male', '15 - 17')]

positive_children = positive_children.reindex(columns = columns)
positive_children.fillna(0, inplace = True)
positive_children.columns = ['_'.join(col) for col in positive_children.columns.values]
positive_children.reset_index(inplace = True)
positive_children.head()

,Commune,positive_Female_< 1,positive_Female_1 - 4,positive_Female_5 - 9,positive_Female_10 - 14,positive_Female_15 - 17,positive_Male_< 1,positive_Male_1 - 4,positive_Male_5 - 9,positive_Male_10 - 14,positive_Male_15 - 17


In [78]:
#Dataset children on arv
hhm_children_on_arv = hhm_positive_children[hhm_positive_children.on_arv == 'yes']

children_on_arv = hhm_children_on_arv.pivot_table(values = 'patient_code', index = 'Commune', 
                                                  columns = ['on_arv', 'gender','age_range'], aggfunc = len, fill_value = 0)

children_on_arv.rename(columns = {'yes': 'on_arv'}, inplace = True)

columns = [('on_arv','Female', '< 1'),
 ('on_arv','Female', '1 - 4'),
 ('on_arv','Female', '5 - 9'),
 ('on_arv','Female', '10 - 14'),
 ('on_arv','Female', '15 - 17'),
 ('on_arv','Male', '< 1'),
 ('on_arv','Male', '1 - 4'),
 ('on_arv','Male', '5 - 9'),
 ('on_arv','Male', '10 - 14'),
 ('on_arv','Male', '15 - 17')]
children_on_arv = children_on_arv.reindex(columns = columns)
children_on_arv.columns = ['_'.join(col) for col in children_on_arv.columns.values]
children_on_arv.fillna(0, inplace = True)
children_on_arv.reset_index(inplace = True)
children_on_arv

#Dataset children not on arv
hhm_children_not_on_arv = hhm_positive_children[hhm_positive_children.on_arv == 'no']

children_not_on_arv = hhm_children_not_on_arv.pivot_table(values = 'patient_code', index = 'Commune',
                                                         columns = ['on_arv','gender','age_range'], aggfunc = len, fill_value = 0)

children_not_on_arv.rename(columns = {'no': 'not_on_arv'}, inplace = True)

columns = [('not_on_arv','Female', '< 1'),
 ('not_on_arv','Female', '1 - 4'),
 ('not_on_arv','Female', '5 - 9'),
 ('not_on_arv','Female', '10 - 14'),
 ('not_on_arv','Female', '15 - 17'),
 ('not_on_arv','Male', '< 1'),
 ('not_on_arv','Male', '1 - 4'),
 ('not_on_arv','Male', '5 - 9'),
 ('not_on_arv','Male', '10 - 14'),
 ('not_on_arv','Male', '15 - 17')]
children_not_on_arv = children_not_on_arv.reindex(columns = columns)
children_not_on_arv.columns = ['_'.join(col) for col in children_not_on_arv.columns.values]
children_not_on_arv.fillna(0, inplace = True)
children_not_on_arv.reset_index(inplace = True)

children_not_on_arv

,Commune,on_arv_Female_< 1,on_arv_Female_1 - 4,on_arv_Female_5 - 9,on_arv_Female_10 - 14,on_arv_Female_15 - 17,on_arv_Male_< 1,on_arv_Male_1 - 4,on_arv_Male_5 - 9,on_arv_Male_10 - 14,on_arv_Male_15 - 17


,Commune,not_on_arv_Female_< 1,not_on_arv_Female_1 - 4,not_on_arv_Female_5 - 9,not_on_arv_Female_10 - 14,not_on_arv_Female_15 - 17,not_on_arv_Male_< 1,not_on_arv_Male_1 - 4,not_on_arv_Male_5 - 9,not_on_arv_Male_10 - 14,not_on_arv_Male_15 - 17


### No HIV reported (Status Unknown)

In [79]:
#Positive children not on HIV Haiti
hhm_positive_children_not_on_hivhaiti = hhm_ff[(hhm_ff.hiv_test_result == 'positif') & (hhm_ff.on_HIVHAITI == 'no') & (hhm_ff.age_in_year <= 17)]
positive_children_not_on_hivhaiti = hhm_positive_children_not_on_hivhaiti.pivot_table(values = 'patient_code', index = 'Commune',
                                                                                     columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len,
                                                                                     fill_value = 0)
# positive_children_not_on_hivhaiti.rename(columns = {'positif': 'No HIV reported (positive_not_on_hivhaiti)'}, inplace = True)
positive_children_not_on_hivhaiti.rename(columns = {'positif': 'No_HIV_reported_Status_Unknown'}, inplace = True)

columns = [('No_HIV_reported_Status_Unknown','Female', '< 1'),
 ('No_HIV_reported_Status_Unknown','Female', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Female', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Female', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Female', '15 - 17'),
 ('No_HIV_reported_Status_Unknown','Male', '< 1'),
 ('No_HIV_reported_Status_Unknown','Male', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Male', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Male', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Male', '15 - 17')]
positive_children_not_on_hivhaiti = positive_children_not_on_hivhaiti.reindex(columns = columns)
positive_children_not_on_hivhaiti.columns = ['_'.join(col) for col in positive_children_not_on_hivhaiti.columns.values]
positive_children_not_on_hivhaiti.fillna(0, inplace = True)
positive_children_not_on_hivhaiti.reset_index(inplace = True)

#Children unknown status
status_unknown = hhm_ff[(~hhm_ff.hiv_test_result.isin(['positif', 'negatif'])) & (hhm_ff.age_in_year <= 17)]
status_unknown.loc[(~status_unknown.hiv_test_result.isin(['positif', 'negatif'])), 'hiv_test_result'] = '---'
children_status_unknown = status_unknown.pivot_table(values = 'patient_code', index = 'Commune', 
                                                     columns = ['hiv_test_result', 'gender','age_range'],
                                                    aggfunc = len, fill_value = 0)
# children_status_unknown.rename(columns = {'---': 'No HIV reported (status Unknown_without positive not on Hiv Haiti)'}, inplace = True)
children_status_unknown.rename(columns = {'---': 'No_HIV_reported_Status_Unknown'}, inplace = True)

columns = [('No_HIV_reported_Status_Unknown','Female', '< 1'),
 ('No_HIV_reported_Status_Unknown','Female', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Female', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Female', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Female', '15 - 17'),
 ('No_HIV_reported_Status_Unknown','Male', '< 1'),
 ('No_HIV_reported_Status_Unknown','Male', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Male', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Male', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Male', '15 - 17')]
children_status_unknown = children_status_unknown.reindex(columns = columns)
children_status_unknown.columns = ['_'.join(col) for col in children_status_unknown.columns.values]
children_status_unknown.fillna(0, inplace = True)
children_status_unknown.reset_index(inplace = True)

#Sum
children_unknown_status = pd.concat([positive_children_not_on_hivhaiti,children_status_unknown], ignore_index=True)
children_unknown_status = children_unknown_status.groupby(["Commune"]).sum()
children_unknown_status.reset_index(inplace = True)
children_unknown_status.head()
children_unknown_status.shape

,Commune,No_HIV_reported_Status_Unknown_Female_< 1,No_HIV_reported_Status_Unknown_Female_1 - 4,No_HIV_reported_Status_Unknown_Female_5 - 9,No_HIV_reported_Status_Unknown_Female_10 - 14,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17
0,Anse D'Hainault,2,3,3,1,1,1,2,3,1,1
1,Anse-Rouge,0,1,4,0,1,1,0,0,2,0
2,Anse-a-Galets,2,1,0,0,1,0,1,0,1,1
3,Anse-a-Veau,3,1,1,1,0,1,3,0,0,0
4,Aquin,1,2,1,2,1,1,3,3,1,0


(48, 11)

In [80]:
#Concatenation _ FINAL DATA _ OVC_HIVSTAT_CHILDREN

# index = ['Commune','positive', 'on_arv', 'not_on_arv', 'negative', 'test not indicated', 'No_HIV_reported_Status_Unknown']
#------------------------------------------------------------------------------------------------------------------------
def ovc_hivstat_children():
    if len(positive_children) != 0 and len(children_on_arv) != 0 and len(children_not_on_arv) != 0 and len(negative_children) != 0 and len(children_unknown_status) != 0:
        ovc_hivstat_children = positive_children.merge(children_on_arv, on = 'Commune', how = 'outer').merge(children_not_on_arv, on = 'Commune', how = 'outer').merge(negative_children, on = 'Commune', how = 'outer').merge(children_unknown_status, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children['test not indicated'] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)        
        return ovc_hivstat_children
    elif len(positive_children) != 0 and len(children_on_arv) == 0 and len(children_not_on_arv) != 0 and len(negative_children) != 0 and len(children_unknown_status) != 0:
        ovc_hivstat_children = positive_children.merge(children_not_on_arv, on = 'Commune', how = 'outer').merge(negative_children, on = 'Commune', how = 'outer').merge(children_unknown_status, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children['children_on_arv','test not indicated'] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)        
        return ovc_hivstat_children
    elif len(positive_children) != 0 and len(children_on_arv) != 0 and len(children_not_on_arv) == 0 and len(negative_children) != 0 and len(children_unknown_status) != 0:
        ovc_hivstat_children = positive_children.merge(children_on_arv, on = 'Commune', how = 'outer').merge(negative_children, on = 'Commune', how = 'outer').merge(children_unknown_status, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children['children_not_on_arv','test not indicated'] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)        
        return ovc_hivstat_children
    elif len(positive_children) == 0 and len(negative_children) != 0 and len(children_unknown_status) != 0:
        ovc_hivstat_children = negative_children.merge(children_unknown_status, on = 'Commune', how = 'outer')
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children[['positive','on arv', 'not on arv','test not indicated']] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)
        return ovc_hivstat_children
    elif len(positive_children) == 0 and len(negative_children) == 0 and len(children_unknown_status) != 0:
        ovc_hivstat_children = children_unknown_status
        ovc_hivstat_children = ovc_hivstat_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_children[['positive','on arv', 'not on arv', 'negative','test not indicated']] = 0
        ovc_hivstat_children.fillna(0, inplace = True)
#         ovc_hivstat_children = ovc_hivstat_children.reindex(columns = index)
        return ovc_hivstat_children
    else:
        return 'HIV test data are not available right now!!'
    
    
#Calling the function
ovc_hivstat_children()

,Commune,negative_Female_< 1,negative_Female_1 - 4,negative_Female_5 - 9,negative_Female_10 - 14,negative_Female_15 - 17,negative_Male_< 1,negative_Male_1 - 4,negative_Male_5 - 9,negative_Male_10 - 14,...,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17,positive,on arv,not on arv,test not indicated
0,Anse D'Hainault,0,2,0,1,0,1,2,1,0,...,1.0,1.0,2.0,3.0,1.0,1.0,0,0,0,0
1,Anse-Rouge,0,0,1,0,0,0,0,1,1,...,1.0,1.0,0.0,0.0,2.0,0.0,0,0,0,0
2,Anse-a-Galets,5,26,25,12,10,4,31,14,18,...,1.0,0.0,1.0,0.0,1.0,1.0,0,0,0,0
3,Anse-a-Veau,0,2,2,2,0,2,5,2,4,...,0.0,1.0,3.0,0.0,0.0,0.0,0,0,0,0
4,Aquin,3,3,9,5,2,5,8,10,3,...,1.0,1.0,3.0,3.0,1.0,0.0,0,0,0,0
5,Arcahaie,6,44,47,33,14,3,32,33,21,...,2.0,19.0,12.0,7.0,5.0,3.0,0,0,0,0
6,Bassin Bleu,1,9,6,3,0,7,7,9,3,...,1.0,2.0,1.0,4.0,0.0,0.0,0,0,0,0
7,Cap-Haitien,9,21,15,8,3,10,19,17,7,...,1.0,6.0,6.0,12.0,7.0,7.0,0,0,0,0
8,Caracol,1,3,5,3,1,0,6,2,3,...,1.0,1.0,3.0,1.0,2.0,1.0,0,0,0,0
9,Carrefour,3,13,19,22,6,9,13,21,23,...,0.0,6.0,6.0,2.0,3.0,0.0,0,0,0,0


# DATIM Other kids

In [81]:
#Creating Datim table for the other children
index_other = hhm_other.pivot_table(values = 'patient_code', index = 'Commune', columns = ['gender', 'age_range'], 
                                    aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')

columns = [('Female', '< 1'),
 ('Female', '1 - 4'),
 ('Female', '5 - 9'),
 ('Female', '10 - 14'),
 ('Female', '15 - 17'),
 ('Female', '18+'),
 ('Male', '< 1'),
 ('Male', '1 - 4'),
 ('Male', '5 - 9'),
 ('Male', '10 - 14'),
 ('Male', '15 - 17'),
 ('Male', '18+')]

index_other = index_other.reindex(columns = columns)
index_other[('Female', '18+')] = 0
index_other[('Male', '18+')] = 0

index_other.reset_index(inplace = True)

index_other

gender                 Commune Female                                 Male  \
age_range                         < 1 1 - 4 5 - 9 10 - 14 15 - 17 18+  < 1   
0              Anse D'Hainault      0     1     1       4       4   0    0   
1                   Anse-Rouge      3     0     0       0       0   0    1   
2                Anse-a-Galets      0     0     2       1       1   0    0   
3                  Anse-a-Veau      0     2     0       0       2   0    0   
4                        Aquin      0     1     2       1       0   0    0   
5                     Arcahaie      1     3     7       5       4   0    5   
6                  Bassin Bleu      0     2     1       1       0   0    1   
7                  Cap-Haitien      1     3     6       5       4   0    0   
8                      Caracol      0     2     0       0       0   0    0   
9                    Carrefour      3     6     4       7       1   0    2   
10                   Cavaillon      2     0     4       1       3   0    5   
11                       Cayes      0     0     2       2       2   0    1   
12          Croix-des-Bouquets      0     0    29      35       1   0    0   
13                      Delmas      3     8    63      63      28   0    4   
14                  Dessalines      0     0     1       2       0   0    1   
15                      Ennery      1     1     0       0       0   0    5   
16                    Gonaives      2     7     1       3       2   0    5   
17                  Gros-Morne      2     2     1       3       3   0    8   
18                      Hinche      0     0     0       0       0   0    0   
19                 Ile-a-Vache      0     0     0       0       1   0    0   
20                     Jeremie      0     9     9      10       7   0    1   
21                    Kenscoff      2     3     3       8       3   0    1   
22                     L'Asile      0     2     0       0       1   0    0   
23                    L'Estere      0     0     0       0       0   0    1   
24                   La Tortue      0     0     0       0       0   0    0   
25                     Leogane      0     1     1       3       1   0    0   
26                 Les Anglais      1     0     0       0       1   0    0   
27                       Limbe      0     0     2       2       2   0    1   
28                   Marmelade      0     1     0       0       0   0    0   
29                       Milot      1     2     1       4       3   0    0   
30                   Miragoane      1     0     2       0       9   0    2   
31               Mont-Organise      0     0     0       0       0   0    0   
32                       Moron      0     0     7      11       2   0    0   
33                 Ouanaminthe      0     1     2       1       1   0    0   
34                 Petit-Goave      0     1     3       5       0   0    0   
35                  Petit-Trou      6     0     0       0       1   0    8   
36              Petite-Riviere      0     0     0       0       0   0    0   
37                      Pignon      3     3     1       4       2   0    1   
38                      Pilate      1     4    11       4       0   0    1   
39              Plaine-du-Nord      1     1     1       2       4   0    1   
40                 Port-Margot      0     1     2       2       2   0    1   
41              Port-au-Prince      3     8    59      62      18   0    7   
42                Port-de-Paix      1     5     3       2       2   0    2   
43         Saint-Louis-du-Nord      0     0     0       3       1   0    0   
44                  Saint-Marc      3     1     8      12       5   0    0   
45               Saint-Raphael      0     1     2       1       0   0    0   
46                     Tabarre      1     6     4      19       5   0    1   
47               Terrier-Rouge      0     0     2       2       0   0    1   
48                Trou-du-Nord      1     1     7      13      11   0    1   
49                    

# OVC_HIVSTAT_OTHER

In [82]:
#Negative other children
hhm_negative_other_children = hhm_other[(hhm_other.hiv_test_result == 'negatif') & (hhm_other.age_in_year <= 17)]

negative_other_children = hhm_negative_other_children.pivot_table(index = 'Commune', values = 'patient_code', 
                                                      columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len, fill_value = 0)

negative_other_children.rename(columns = {'negatif': 'negative'}, inplace = True)

columns = [('negative','Female', '< 1'),
 ('negative','Female', '1 - 4'),
 ('negative','Female', '5 - 9'),
 ('negative','Female', '10 - 14'),
 ('negative','Female', '15 - 17'),
 ('negative','Male', '< 1'),
 ('negative','Male', '1 - 4'),
 ('negative','Male', '5 - 9'),
 ('negative','Male', '10 - 14'),
 ('negative','Male', '15 - 17')]

negative_other_children = negative_other_children.reindex(columns = columns)
negative_other_children.columns = ['_'.join(col) for col in negative_other_children.columns.values]
negative_other_children.reset_index(inplace = True)

negative_other_children.head()

,Commune,negative_Female_< 1,negative_Female_1 - 4,negative_Female_5 - 9,negative_Female_10 - 14,negative_Female_15 - 17,negative_Male_< 1,negative_Male_1 - 4,negative_Male_5 - 9,negative_Male_10 - 14,negative_Male_15 - 17
0,Anse-Rouge,3,0,0,0,0,1,0,0,0,0
1,Anse-a-Galets,0,0,2,1,1,0,0,1,2,4
2,Anse-a-Veau,0,1,0,0,0,0,0,0,0,0
3,Aquin,0,0,1,1,0,0,0,0,3,0
4,Arcahaie,0,1,3,3,3,0,1,2,2,1


In [83]:
#Positive other children
hhm_positive_other_children = hhm_other[(hhm_other.hiv_test_result == 'positif') & (hhm_other.on_HIVHAITI == 'yes') & (hhm_other.age_in_year <= 17)].drop_duplicates('full_code_patient_menage')
hhm_positive_other_children = hhm_positive_other_children.merge(regime[['full_code_patient_menage', 'arv_start_date']], 
                                                    on = 'full_code_patient_menage', how = 'left')
hhm_positive_other_children['on_arv'] = hhm_positive_other_children.arv_start_date.map(on_arv)
positive_other_children = hhm_positive_other_children.pivot_table(values = 'patient_code', index = 'Commune', 
                                                      columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len, fill_value = 0)
positive_other_children.rename(columns = {'positif': 'positive'}, inplace = True)

columns = [('positive','Female', '< 1'),
 ('positive','Female', '1 - 4'),
 ('positive','Female', '5 - 9'),
 ('positive','Female', '10 - 14'),
 ('positive','Female', '15 - 17'),
 ('positive','Male', '< 1'),
 ('positive','Male', '1 - 4'),
 ('positive','Male', '5 - 9'),
 ('positive','Male', '10 - 14'),
 ('positive','Male', '15 - 17')]

positive_other_children = positive_other_children.reindex(columns = columns)
positive_other_children.fillna(0, inplace = True)
positive_other_children.columns = ['_'.join(col) for col in positive_other_children.columns.values]
positive_other_children.reset_index(inplace = True)

positive_other_children

,Commune,positive_Female_< 1,positive_Female_1 - 4,positive_Female_5 - 9,positive_Female_10 - 14,positive_Female_15 - 17,positive_Male_< 1,positive_Male_1 - 4,positive_Male_5 - 9,positive_Male_10 - 14,positive_Male_15 - 17


In [84]:
#Dataset children on arv
hhm_other_children_on_arv = hhm_positive_other_children[hhm_positive_other_children.on_arv == 'yes']
other_children_on_arv = hhm_other_children_on_arv.pivot_table(values = 'patient_code', index = 'Commune', 
                                                  columns = ['on_arv', 'gender','age_range'], aggfunc = len, fill_value = 0)
other_children_on_arv.rename(columns = {'yes': 'on_arv'}, inplace = True)

columns = [('on_arv','Female', '< 1'),
 ('on_arv','Female', '1 - 4'),
 ('on_arv','Female', '5 - 9'),
 ('on_arv','Female', '10 - 14'),
 ('on_arv','Female', '15 - 17'),
 ('on_arv','Male', '< 1'),
 ('on_arv','Male', '1 - 4'),
 ('on_arv','Male', '5 - 9'),
 ('on_arv','Male', '10 - 14'),
 ('on_arv','Male', '15 - 17')]
other_children_on_arv = other_children_on_arv.reindex(columns = columns)
other_children_on_arv.columns = ['_'.join(col) for col in other_children_on_arv.columns.values]
other_children_on_arv.reset_index(inplace = True)

other_children_on_arv

#Dataset children not on arv
hhm_other_children_not_on_arv = hhm_positive_other_children[hhm_positive_children.on_arv == 'no']

other_children_not_on_arv = hhm_other_children_not_on_arv.pivot_table(values = 'patient_code', index = 'Commune',
                                                         columns = ['on_arv', 'gender','age_range'], aggfunc = len, fill_value = 0)

other_children_not_on_arv.rename(columns = {'no': 'not_on_arv'}, inplace = True)

columns = [('not_on_arv','Female', '< 1'),
 ('not_on_arv','Female', '1 - 4'),
 ('not_on_arv','Female', '5 - 9'),
 ('not_on_arv','Female', '10 - 14'),
 ('not_on_arv','Female', '15 - 17'),
 ('not_on_arv','Male', '< 1'),
 ('not_on_arv','Male', '1 - 4'),
 ('not_on_arv','Male', '5 - 9'),
 ('not_on_arv','Male', '10 - 14'),
 ('not_on_arv','Male', '15 - 17')]
other_children_not_on_arv = other_children_not_on_arv.reindex(columns = columns)
other_children_not_on_arv.columns = ['_'.join(col) for col in other_children_not_on_arv.columns.values]
other_children_not_on_arv.reset_index(inplace = True)
other_children_not_on_arv

,Commune,on_arv_Female_< 1,on_arv_Female_1 - 4,on_arv_Female_5 - 9,on_arv_Female_10 - 14,on_arv_Female_15 - 17,on_arv_Male_< 1,on_arv_Male_1 - 4,on_arv_Male_5 - 9,on_arv_Male_10 - 14,on_arv_Male_15 - 17


,Commune,not_on_arv_Female_< 1,not_on_arv_Female_1 - 4,not_on_arv_Female_5 - 9,not_on_arv_Female_10 - 14,not_on_arv_Female_15 - 17,not_on_arv_Male_< 1,not_on_arv_Male_1 - 4,not_on_arv_Male_5 - 9,not_on_arv_Male_10 - 14,not_on_arv_Male_15 - 17


In [85]:
#Positive children not on HIV Haiti
hhm_positive_other_children_not_on_hivhaiti = hhm_other[(hhm_other.hiv_test_result == 'positif') & (hhm_other.on_HIVHAITI == 'no') & (hhm_other.age_in_year <= 17)]
positive_other_children_not_on_hivhaiti = hhm_positive_other_children_not_on_hivhaiti.pivot_table(values = 'patient_code', index = 'Commune',
                                                                                     columns = ['hiv_test_result', 'gender','age_range'], aggfunc = len,
                                                                                     fill_value = 0)
# positive_children_not_on_hivhaiti.rename(columns = {'positif': 'No HIV reported (positive_not_on_hivhaiti)'}, inplace = True)
positive_other_children_not_on_hivhaiti.rename(columns = {'positif': 'No_HIV_reported_Status_Unknown'}, inplace = True)

columns = [('No_HIV_reported_Status_Unknown','Female', '< 1'),
 ('No_HIV_reported_Status_Unknown','Female', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Female', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Female', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Female', '15 - 17'),
 ('No_HIV_reported_Status_Unknown','Male', '< 1'),
 ('No_HIV_reported_Status_Unknown','Male', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Male', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Male', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Male', '15 - 17')]
positive_other_children_not_on_hivhaiti = positive_other_children_not_on_hivhaiti.reindex(columns = columns)
positive_other_children_not_on_hivhaiti.columns = ['_'.join(col) for col in positive_other_children_not_on_hivhaiti.columns.values]
positive_other_children_not_on_hivhaiti.fillna(0, inplace = True)
positive_other_children_not_on_hivhaiti.reset_index(inplace = True)

#Children unknown status
other_status_unknown = hhm_other[(~hhm_other.hiv_test_result.isin(['positif', 'negatif'])) & (hhm_other.age_in_year <= 17)]
other_status_unknown.loc[(~other_status_unknown.hiv_test_result.isin(['positif', 'negatif'])), 'hiv_test_result'] = '---'
other_children_status_unknown = other_status_unknown.pivot_table(values = 'patient_code', index = 'Commune', 
                                                     columns = ['hiv_test_result', 'gender','age_range'],
                                                    aggfunc = len, fill_value = 0)
# other_children_status_unknown.rename(columns = {'---': 'No HIV reported (status Unknown_without positive not on Hiv Haiti)'}, inplace = True)
other_children_status_unknown.rename(columns = {'---': 'No_HIV_reported_Status_Unknown'}, inplace = True)

columns = [('No_HIV_reported_Status_Unknown','Female', '< 1'),
 ('No_HIV_reported_Status_Unknown','Female', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Female', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Female', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Female', '15 - 17'),
 ('No_HIV_reported_Status_Unknown','Male', '< 1'),
 ('No_HIV_reported_Status_Unknown','Male', '1 - 4'),
 ('No_HIV_reported_Status_Unknown','Male', '5 - 9'),
 ('No_HIV_reported_Status_Unknown','Male', '10 - 14'),
 ('No_HIV_reported_Status_Unknown','Male', '15 - 17')]
other_children_status_unknown = other_children_status_unknown.reindex(columns = columns)
other_children_status_unknown.columns = ['_'.join(col) for col in other_children_status_unknown.columns.values]
other_children_status_unknown.fillna(0, inplace = True)
other_children_status_unknown.reset_index(inplace = True)

#Sum
other_unknown_status_children = pd.concat([positive_other_children_not_on_hivhaiti, other_children_status_unknown], ignore_index=True)
other_unknown_status_children = other_unknown_status_children.groupby(["Commune"]).sum()
other_unknown_status_children.reset_index(inplace = True)
other_unknown_status_children.head()
other_unknown_status_children.shape

,Commune,No_HIV_reported_Status_Unknown_Female_< 1,No_HIV_reported_Status_Unknown_Female_1 - 4,No_HIV_reported_Status_Unknown_Female_5 - 9,No_HIV_reported_Status_Unknown_Female_10 - 14,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17
0,Anse D'Hainault,0,1,1.0,4,4.0,0,0.0,1.0,1,1
1,Anse-a-Veau,0,1,0.0,0,2.0,0,1.0,0.0,3,0
2,Aquin,0,1,1.0,0,0.0,0,0.0,1.0,0,0
3,Arcahaie,1,2,4.0,2,1.0,5,1.0,0.0,2,0
4,Bassin Bleu,0,0,1.0,1,0.0,0,1.0,1.0,0,1


(43, 11)

In [86]:
# index2 = ['Commune','positive', 'on_arv', 'not_on_arv', 'negative', 'test not indicated', 'No HIV reported (Status Unknown)']

#--------------------------------------------------------------------------------------------------------------------
#Concatenation _ OVC_HIVSTAT_OTHER_CHILDREN

def ovc_hivstat_other_children():
    if len(positive_other_children) != 0 and len(other_children_on_arv) != 0 and len(other_children_not_on_arv) != 0 and len(negative_other_children) != 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = positive_other_children.merge(other_children_on_arv, on = 'Commune', how = 'outer').merge(other_children_not_on_arv, on = 'Commune', how = 'outer').merge(negative_other_children, on = 'Commune', how = 'outer').merge(other_unknown_status_children, on = 'Commune', how = 'outer')
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children['test not indicated'] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)        
        return ovc_hivstat_other_children
    elif len(positive_other_children) != 0 and len(other_children_on_arv) == 0 and len(other_children_not_on_arv) != 0 and len(negative_other_children) != 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = positive_other_children.merge(other_children_not_on_arv, on = 'Commune', how = 'outer').merge(negative_other_children, on = 'Commune', how = 'outer').merge(other_unknown_status_children, on = 'Commune', how = 'outer')
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children['other_children_on_arv','test not indicated'] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)        
        return ovc_hivstat_other_children
    elif len(positive_other_children) != 0 and len(other_children_on_arv) != 0 and len(other_children_not_on_arv) == 0 and len(negative_other_children) != 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = positive_other_children.merge(other_children_on_arv, on = 'Commune', how = 'outer').merge(negative_other_children, on = 'Commune', how = 'outer').merge(other_unknown_status_children, on = 'Commune', how = 'outer')
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children['other_children_not_on_arv','test not indicated'] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)        
        return ovc_hivstat_other_children
    elif len(positive_other_children) == 0 and len(negative_other_children) != 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = negative_other_children.merge(other_unknown_status_children, on = 'Commune', how = 'outer')
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children[['positive','on arv', 'not on arv','test not indicated']] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)
        return ovc_hivstat_other_children
    elif len(positive_other_children) == 0 and len(negative_other_children) == 0 and len(other_unknown_status_children) != 0:
        ovc_hivstat_other_children = other_unknown_status_children
        ovc_hivstat_other_children = ovc_hivstat_other_children.sort_values('Commune').reset_index(drop=True)
        ovc_hivstat_other_children[['positive','on arv', 'not on arv', 'negative','test not indicated']] = 0
        ovc_hivstat_other_children.fillna(0, inplace = True)
#         ovc_hivstat_other_children = ovc_hivstat_other_children.reindex(columns = index2)
        return ovc_hivstat_other_children
    else:
        return 'HIV test data are not available right now!!'


#Calling the function
ovc_hivstat_other_children()
        

,Commune,negative_Female_< 1,negative_Female_1 - 4,negative_Female_5 - 9,negative_Female_10 - 14,negative_Female_15 - 17,negative_Male_< 1,negative_Male_1 - 4,negative_Male_5 - 9,negative_Male_10 - 14,...,No_HIV_reported_Status_Unknown_Female_15 - 17,No_HIV_reported_Status_Unknown_Male_< 1,No_HIV_reported_Status_Unknown_Male_1 - 4,No_HIV_reported_Status_Unknown_Male_5 - 9,No_HIV_reported_Status_Unknown_Male_10 - 14,No_HIV_reported_Status_Unknown_Male_15 - 17,positive,on arv,not on arv,test not indicated
0,Anse D'Hainault,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,1.0,1.0,1.0,0,0,0,0
1,Anse-Rouge,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2,Anse-a-Galets,0.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
3,Anse-a-Veau,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,1.0,0.0,3.0,0.0,0,0,0,0
4,Aquin,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,0
5,Arcahaie,0.0,1.0,3.0,3.0,3.0,0.0,1.0,2.0,2.0,...,1.0,5.0,1.0,0.0,2.0,0.0,0,0,0,0
6,Bassin Bleu,0.0,2.0,0.0,0.0,0.0,1.0,3.0,2.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0,0,0,0
7,Cap-Haitien,1.0,0.0,2.0,2.0,0.0,0.0,1.0,3.0,0.0,...,4.0,0.0,2.0,1.0,1.0,3.0,0,0,0,0
8,Caracol,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0,0,0,0
9,Carrefour,1.0,4.0,0.0,5.0,0.0,0.0,1.0,1.0,0.0,...,1.0,2.0,0.0,3.0,1.0,1.0,0,0,0,0


# Caregivers to add to other children (OVC component file)

In [88]:
#Caregivers
hhm_club.rename(columns = {'form.household_mother.caregiver_yes_no':'caregiver_yes_no'}, inplace = True)
hhm_caregiver = hhm_club[((hhm_club['caregiver_yes_no'] == str(1)) | (hhm_club['caregiver_yes_no'] == 'oui')) & (hhm_club['age_in_year'] >= 18)]
hhm_caregiver.loc[(hhm_caregiver.caregiver_yes_no == str(1)), 'caregiver_yes_no'] = 'yes'
hhm_caregiver = hhm_caregiver.drop_duplicates('patient_code').reset_index(drop = True)
print(f"Number of caregivers: {hhm_caregiver.shape[0]}")

Number of caregivers: 1616


In [89]:
hhm_caregiver.to_excel('Caregivers_PTME_FY22Q4_10122022.xlsx', index = False)

In [90]:
#Clustering by commune
caregiver = hhm_caregiver.pivot_table(index = 'Commune', values = 'patient_code', columns = ['gender', 'caregiver_yes_no'],
                                      aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')
caregiver.drop('Total', axis = 1, inplace = True)
caregiver.rename(columns = {'yes': 'Caregiver 18+'}, inplace = True)
caregiver.reset_index(inplace = True)
caregiver

gender,Commune,Female,Male
caregiver_yes_no,,Caregiver 18+,Caregiver 18+
0,Anse D'Hainault,2,9
1,Anse-Rouge,3,2
2,Anse-a-Galets,0,7
3,Anse-a-Veau,10,8
4,Aquin,7,16
5,Arcahaie,1,12
6,Bassin Bleu,6,10
7,Cap-Haitien,13,40
8,Caracol,2,6


# Direct beneficiaries

In [91]:
OVC['age_range'] = OVC['age'].map(age_range)
OVC['gender'] = 'Female'
OVC_serv = OVC[OVC.age_range != '< 1']
serv_mother = OVC_serv.pivot_table(index = 'Commune', values = 'patient_code', columns = ['gender', 'age_range'],
                             aggfunc = len, fill_value = 0, margins = True, margins_name = 'Total')
serv_mother.drop('Total', axis = 1, inplace = True)
serv_mother

gender               Female              
age_range           10 - 14 15 - 17   18+
Commune                                  
Anse D'Hainault           0       0    20
Anse-Rouge                0       0     8
Anse-a-Galets             0       0    39
Anse-a-Veau               0       1    19
Aquin                     0       1    31
Arcahaie                  0       3   174
Bassin Bleu               0       0    39
Cap-Haitien               0       1   105
Capotille                 0       0     1
Caracol                   0       0    12
Carrefour                 0       0    87
Cavaillon                 0       0    27
Cayes                     0       1    62
Cite-Soleil               0       0     1
Croix-des-Bouquets        0       2   200
Delmas                    0       3   358
Dessalines                0       1   154
Ennery                    0       2    58
Gonaives                  0       1   199
Gressier                  0       1     9
Gros-Morne                0       2   152
Hinche                    0       0     9
Ile-a-Vache               0       0     8
Jean-Rabel                0       0    12
Jeremie                   0       0   101
Kenscoff                  0       0    59
L'Asile                   0       0    27
L'Estere                  0       1    53
La Tortue                 0       1     4
Leogane                   0       2    38
Les Anglais               0       0     5
Limbe                     0       1    27
Marmelade                 0       0    79
Milot                     0       0    58
Miragoane                 0       0    70
Mombin-Crochu             0       0     5
Mont-Organise             0       0     9
Moron                     0       0    14
Ouanaminthe               0       1    35
Petit-Goave               0       0    77
Petit-Trou                0       1    11
Petite-Riviere            0       0    47
Pignon                    0       0    60
Pilate                    0       0    69
Plaine-du-Nord            0       1    67
Port-Margot               0       0    20
Port-au-Prince            0       0   432
Port-de-Paix              0       0   138
Saint-Louis-du-Nord       0       2    50
Saint-Marc                1       2   134
Saint-Raphael             0       1    22
Tabarre                   0       0   107
Terrier-Rouge             0       0    28
Trou-du-Nord              0       0    94
Verrettes                 0       0     5
Total                     1      32  3729

# Export

In [92]:
with pd.ExcelWriter('DATIM_OVC_Mothers.xlsx', engine = 'openpyxl') as writer:
    serv_mother.to_excel(writer, sheet_name = 'Bénéficiaires directes', na_rep = 0)
    index_ff.to_excel(writer, sheet_name = 'DATIM_Direct', na_rep = 0)
    ovc_hivstat_children().to_excel(writer, sheet_name = 'HIV_STAT_Direct', na_rep = 0, index = False)
    index_other.to_excel(writer, sheet_name = 'DATIM_Other', na_rep = 0)
    ovc_hivstat_other_children().to_excel(writer, sheet_name = 'HIV_STAT_Other', na_rep = 0, index = False)
    caregiver.to_excel(writer, sheet_name = 'Caregivers_18+', na_rep = 0)

In [94]:
#Exporting the list of mothers aged less than 18
ovc_data_18 = OVC_serv[OVC_serv['age'] < 18]
ovc_data_18.to_excel('OVC_Mother_Q4FY22_less_18.xlsx', index = False)
ovc_data_18.shape[0]

33

In [95]:
OVC_serv.to_excel('Mother_served_FY22Q4_10122022.xlsx', index = False)

In [96]:
ovc_18 = OVC_serv[OVC_serv['age_range'] != '18+']
ovc_18

,patient_code,club_q1,club_q2,on_arv,Commune,age,age_range,gender
100,ARC/SADA/ST01665,no,no,yes,Arcahaie,17,15 - 17,Female
281,CAP/CDSF/ST03124,yes,yes,yes,Cap-Haitien,16,15 - 17,Female
342,CAP/CMCD/ST01507,no,no,yes,Plaine-du-Nord,17,15 - 17,Female
371,CAP/CMSO/ST02274,yes,yes,yes,Ouanaminthe,17,15 - 17,Female
521,CAY/CDSL/ST02088,yes,yes,yes,Cayes,17,15 - 17,Female
624,ENN/HTLO/ST00656,no,no,yes,Ennery,15,15 - 17,Female
655,FDB/HOSB/ST03442,no,yes,yes,Aquin,16,15 - 17,Female
895,GON/CDIR/ST01709,yes,yes,yes,Gonaives,17,15 - 17,Female
981,GON/CSFA/ST01139,no,no,yes,Ennery,16,15 - 17,Female
1102,GON/HCRE/ST00386,no,yes,yes,L'Estere,17,15 - 17,Female


In [97]:
pos_mother = ovc_18.pivot_table(index = 'Commune', columns = ['age_range'], values = 'patient_code',
                               aggfunc = len, fill_value = 0)
pos_mother.rename(columns = {
    '10 - 14': 'positive_Female_10 - 14',
    '15 - 17': 'positive_Female_15 - 17'
}, inplace = True)
pos_mother.fillna(0, inplace = True)
pos_mother.reset_index(inplace = True)
pos_mother

age_range,Commune,positive_Female_10 - 14,positive_Female_15 - 17
0,Anse-a-Veau,0,1
1,Aquin,0,1
2,Arcahaie,0,3
3,Cap-Haitien,0,1
4,Cayes,0,1
5,Croix-des-Bouquets,0,2
6,Delmas,0,3
7,Dessalines,0,1
8,Ennery,0,2
9,Gonaives,0,1


In [98]:
ovc_18_hiv = ovc_18.pivot_table(index = 'Commune', values = 'patient_code', columns = ['on_arv', 'age_range'], aggfunc = len,
                                fill_value = 0)
ovc_18_hiv.rename(columns = {'yes': 'on_arv',
                            'no': 'not_on_arv'}, inplace = True)
ovc_18_hiv.columns = ['_'.join(col) for col in ovc_18_hiv.columns.values]
ovc_18_hiv.rename(columns = {'not_on_arv_15 - 17': 'not_on_arv_Female_15 - 17',
                            'on_arv_10 - 14': 'on_arv_Female_10 - 14',
                            'on_arv_15 - 17': 'on_arv_Female_15 - 17'}, inplace = True)
ovc_18_hiv.reset_index(inplace = True)
ovc_18_hiv

,Commune,not_on_arv_Female_15 - 17,on_arv_Female_10 - 14,on_arv_Female_15 - 17
0,Anse-a-Veau,0,0,1
1,Aquin,0,0,1
2,Arcahaie,0,0,3
3,Cap-Haitien,0,0,1
4,Cayes,0,0,1
5,Croix-des-Bouquets,0,0,2
6,Delmas,0,0,3
7,Dessalines,0,0,1
8,Ennery,0,0,2
9,Gonaives,0,0,1


In [99]:
ovc_18_hiv.to_excel('HIVSTAT_mothers.xlsx', index = False)

## Closing the connections...

In [100]:
driver.quit()
engine.dispose()

                                                                                                            The end